# Cruzando múltiplas bases


Uma outra situação bastante comum é que as características de uma determinada observação estejam espalhadas em múltiplas bases.

Esse é o padrão adotado em bancos de dados relacionais, por exemplo, onde uma mesma observação pode ter suas características espalhadas em diferentes bases.

Para **cruzar** esses dados, precisamos de operações que unam as bases a partir de identificadores em comum.

No exemplo a seguir, vamos analisar o cancelamento do semestre por parte dos alunos da UFRN.

## Entendendo os dados

Neste exemplo, vamos lidar com três bases de dados:
- **componentes**, que representam disciplinas existentes em estruturas curriculares de cursos da UFRN;
- **turmas**, que representam ofertas dessas disciplinas em determinados períodos;
- **matrículas**, que listam os discentes matriculados nessas turmas.

Vamos começar conhecendo a base de componentes:

In [0]:
import pandas as pd

In [0]:
componentes = pd.read_csv(
                        "http://dados.ufrn.br/dataset/3fea67e8-6916-4ed0-aaa6-9a8ca06a9bdc/resource/9a3521d2-4bc5-4fda-93f0-f701c8a20727/download/componentes-curriculares-presenciais.csv", 
                        sep=";"
                        )
componentes

Nesta base, as informações que podem nos ser úteis são o identificador do componente, seu nome e sua unidade responsável.

In [0]:
componentes_reduzido = componentes[["id_componente","nome","unidade_responsavel"]]
componentes_reduzido

Antes de prosseguirmos, vamos nos assegurar que não haja dados faltando:

In [0]:
componentes_reduzido.isnull().sum()

Como há apenas um dado faltando, podemos ver se há uma indicação de como preencher esse dado:

In [0]:
componentes_reduzido[componentes_reduzido["nome"].isnull()]

Uma opção aqui seria entrar em contato com a Superintendência de Informática da UFRN, curadora da base de dados.

No entanto, para esta análise vamos excluir essa observação:

In [0]:
componentes_reduzido = componentes_reduzido[~componentes_reduzido["nome"].isnull()]

Vamos agora conferir as bases de turmas, que são disponibilizadas por período:

In [0]:
csv_turmas = {
    "2018.2": "http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/77fe7603-0e71-4e21-8cd4-cb823353023f/download/turmas-2018.2.csv",
    "2018.1": "http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/3ae16138-4214-4a30-ac2d-6cffd6237031/download/turmas-2018.1.csv",
    "2017.2": "http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/01fe7343-fdf0-4a67-b915-2386b7c2fecb/download/turmas-2017.2.csv",
    "2017.1": "http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/5e77d066-d506-45eb-a21e-76aa79616fef/download/turmas-2017.1.csv",
    "2016.2": "http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/5e8e3228-7f22-40a2-9efd-561c44844567/download/turmas-2016.2.csv",
    "2016.1": "http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/322d9977-ba15-47f1-8216-75a1ca78e197/download/turmas-2016.1.csv",
    "2015.2": "http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/7c59621c-4a8b-49d4-b319-83cfea9bdf28/download/turmas-2015.2.csv",
    "2015.1": "http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/4d5aee5a-00b0-4ed6-a4be-59fa77a56797/download/turmas-2015.1.csv",
    "2014.2": "http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/2c69547b-920f-4ec2-92c0-3fbc19512165/download/turmas-2014.2.csv",
    "2014.1": "http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/e6e4144f-4042-4fdc-84e0-76e9ec27ae7c/download/turmas-2014.1.csv",
}

In [0]:
dados_turmas = pd.concat(pd.read_csv(csv_turmas[url], sep=";") for url in csv_turmas)
dados_turmas

Estranhamente, as primeiras observações revelam que há múltiplas observações com o mesmo identificador de turma.

Isso não é comum em bancos de dados relacionais, uma vez que cada base costuma ter um identificador único por observação.

Quando isto acontece, temos a indicação de que a identificação de uma observação depende de múltiplas características.

Vamos isolar um caso assim para ver a diferença entre as observações:

In [0]:
dados_turmas.query('id_turma == 57612672')

Nota-se que a diferença entre as observações está nos campos `siape`, `matricula_docente_externo` e `ch_dedicada_periodo`.

Pesquisando o contexto da UFRN, isso indica que a mesma turma é registrada tantas vezes quantos forem os professores que a estejam lecionando.

Para nossa análise, isto não é interessante, então vamos simplificar esta base removendo as entradas duplicadas com o método `drop_duplicates()`, aplicado após ordenarmos os dados pela coluna `id_turma`:

In [0]:
dados_turmas = dados_turmas.sort_values("id_turma").drop_duplicates(subset="id_turma")
dados_turmas

Vamos dar uma olhada em valores faltando:

In [0]:
dados_turmas.isnull().sum()

😅

Com tantos valores faltando, nossa melhor alternativa é remover as características que estejam incompletas.

Fazemos isso usando o método `drop_na()`, informando que queremos remover colunas (`axis=1`):

In [0]:
dados_turmas = dados_turmas.dropna(axis=1)
dados_turmas

## Cruzando dados de disciplinas e turmas



Até aqui, temos uma grande quantidade de dados à nossa disposição, com 38.276 disciplinas e 91.483 turmas. 

Fazendo um recorte da nossa análise, vamos começar por  disciplinas obrigatórias do Bacharelado em Tecnologia da Informação (BTI):

In [0]:
lista_obrigatórias = [
                      "ANÁLISE COMBINATÓRIA",
                      "CÁLCULO DIFERENCIAL E INTEGRAL I",
                      "ESTRUTURA DE DADOS BÁSICAS I",
                      "ESTRUTURAS DE DADOS BÁSICAS II",
                      "FUNDAMENTOS MATEMÁTICOS DA COMPUTAÇÃO I",
                      "FUNDAMENTOS MATEMÁTICOS DA COMPUTAÇÃO II",
                      "GEOMETRIA EUCLIDIANA",
                      "INTRODUÇÃO ÀS TÉCNICAS DE PROGRAMAÇÃO",
                      "LINGUAGEM DE PROGRAMAÇÃO I",
                      "LINGUAGEM DE PROGRAMAÇÃO II",
                      "MATEMÁTICA ELEMENTAR",
                      "PENSAMENTO COMPUTACIONAL",
                      "PRÁTICAS DE LEITURA EM INGLÊS",
                      "PRÁTICAS DE LEITURA E ESCRITA EM PORTUGUÊS I",
                      "PRÁTICAS DE LEITURA E ESCRITA EM PORTUGUÊS II",
                      "PROBABILIDADE",
                      "TECNOLOGIA DA INFORMAÇÃO E SOCIEDADE",
                      "VETORES E GEOMETRIA ANALÍTICA",
                      ]

unidades_acadêmicas = ["INSTITUTO METROPOLE DIGITAL", "DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA"]
condição_nome = f"nome in {lista_obrigatórias}"
condição_unidade = f"unidade_responsavel in {unidades_acadêmicas}"
componentes_bti_obrigatórios = componentes_reduzido.query(f"{condição_nome} and {condição_unidade}")
componentes_bti_obrigatórios

Para unir os dados de turmas e componentes, vamos usar o método `merge()`, que recebe dois dataframes e a indicação dos identificadores que devem ser usados para o cruzamento: 

In [0]:
turmas_obrigatórias = pd.merge(componentes_bti_obrigatórios, dados_turmas, left_on="id_componente", right_on="id_componente_curricular")
turmas_obrigatórias

Revendo este código:
- `núcleo_comum` é considerado o dataframe à esquerda da união e seu identificador é informado usando o argumento `left_on="id_componente"`
- `data_turmas` é considerado o dataframe à direita da união e seu identificador é informado usando o argumento `right_on="id_componente_curricular"`

Note que agora temos um dataframe consideravelmente menor, com apenas 341 observações.

Isto acontece porque o método `merge()` adota como padrão a **união interna**, que preserva apenas as observações onde os identificadores dos dataframes originais são iguais. 

## Cruzando dados de turmas e de matrículas

O último dado que precisamos coletar é o de matrículas de discentes em turmas.

Apesar do dado para o primeiro período de 2019 estar disponível, vamos coletar apenas os dados até 2018 já que não há dados de turmas ainda para 2019.

**Observação:** a execução da coleta a seguir pode demorar um pouco.

In [0]:
csv_matrículas = {
    "2018.2": "http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/0bfcaf6a-4424-4983-8ba8-d330350a8fbe/download/matricula-componente-20182.csv",
    "2018.1": "http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/3c1feba4-ced1-466e-8e94-a040224a51dc/download/matricula-componente-20181.csv",
    "2017.2": "http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/55dfe713-ff7c-4fa8-8d1d-d4294a025bff/download/matricula-componente-20172.csv",
    "2017.1": "http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/79071c21-e32c-438f-b930-d1b6ccc02ec2/download/matricula-componente-20171.csv",
    "2016.2": "http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/f6179838-b619-4d7d-af9c-18c438b80dd4/download/matriculas-de-2016.2.csv",
    "2016.1": "http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/4778d3ce-8898-46a8-a623-ee6a480a2980/download/matriculas-de-2016.1.csv",
    "2015.2": "http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/baa6c8b4-2072-417f-b238-c028ccc8c14b/download/matriculas-de-2015.2.csv",
    "2015.1": "http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/9e7ba1c2-f92d-4b9c-9e91-3b026ecdf913/download/matriculas-de-2015.1.csv",
    "2014.2": "http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/e974792c-b557-470c-bf3d-ede7d5b5e6a6/download/matricula-componente-20142.csv",
    "2014.1": "http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/7081446d-39f9-4374-ad0b-86ecab97e569/download/matricula-componente-20141.csv",
}

In [0]:
dados_matrículas = pd.concat(pd.read_csv(csv_matrículas[url], sep=";") for url in csv_matrículas)
dados_matrículas

6.943.310 observações! 😱

Vamos repetir os procedimentos que fizemos anteriormente, começando pela limpeza de dados faltando

In [0]:
dados_matrículas.isnull().sum()

In [0]:
dados_matrículas = dados_matrículas.dropna(axis=1)
dados_matrículas

Vamos verificar se também há dados repetidos neste dataframe:

In [0]:
dados_matrículas.groupby(["discente","id_turma","descricao"]).size()

Note que um mesmo discente, apesar de aprovado, aparece como três observações para uma única turma.

Isto indica que uma observação neste dataset é o registro de desempenho de um discente em apenas uma unidade.

Vamos então reduzir este dataframe, eliminando dados repetidos:

In [0]:
dados_matrículas = dados_matrículas.sort_values(["discente","id_turma","descricao"]).drop_duplicates(subset=["discente","id_turma","descricao"])
dados_matrículas

Note que tanto a ordenação como a remoção de dados repetidos foi feita considerando a tupla `"discente","id_turma","descricao"`, já que só queremos considerar repetidas observações que representem múltiplas unidades de um mesmo discente em uma mesma turma.

Assim, conseguimos reduzir a quantidade de observações neste dataframe para "apenas" 2.410.123. 

Para finalizar, vamos cruzar os dados de matrículas com os dados de turmas, usando como identificador a característica `id_turma`:

In [0]:
matrículas_obrigatórias = pd.merge(dados_matrículas, turmas_obrigatórias, on="id_turma")
matrículas_obrigatórias

Após a união interna, nosso dataframe contém apenas 23.933 observações, referentes a discentes que cursaram as disciplinas do nosso recorte.

Vamos persistir esses dados:

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
matrículas_obrigatórias.to_csv("/content/drive/My Drive/obrigatórias-bti-2014-2018.csv", index=False)

## Analisando os resultados dos discentes

Vamos passar a analisar os resultados obtidos pelos discentes nestas disciplinas:

In [0]:
matrículas_obrigatórias.descricao.value_counts()

Entender todos os possíveis resultados exigiria uma boa documentação do dataset, o que ainda não está disponível.

Para efeito desta análise, podemos descartar o caso de indeferimento, já que neste caso o aluno não chegou a cursar a disciplina:

In [0]:
lista_não_cursou = [
                    "CUMPRIU",
                    "DESISTENCIA",
                    "DISPENSADO",
                    "EXCLUIDA",
                    "INDEFERIDO",
                    ]
condição_tentativa = f"not descricao in {lista_não_cursou}"
matrículas_obrigatórias = matrículas_obrigatórias.query(condição_tentativa)

In [0]:
matrículas_obrigatórias.descricao.value_counts()

Chama a atenção a quantidade de possibilidades de reprovação prevista no regulamento dos cursos de graduação da UFRN.

Para simplificar nossa análise, vamos considerar apenas as possibilidades que tiveram um número razoável de casos:

In [0]:
reprovações = ["REPROVADO POR NOTA","REPROVADO POR NOTA E FALTA","REPROVADO POR FALTAS"]
condição = f"not descricao in {reprovações}"
matrículas_obrigatórias = matrículas_obrigatórias.query(condição)

Vamos agora discriminar o resultado por disciplina:

In [0]:
agregado_obrigatórias = matrículas_obrigatórias.groupby(["nome","descricao"]).size()
agregado_obrigatórias

In [0]:
percentual_obrigatórias = pd.crosstab(matrículas_obrigatórias["nome"], matrículas_obrigatórias["descricao"], normalize="index")
percentual_obrigatórias

Como são muitos valores possíveis para analisarmos, vamos usar o `catplot` da biblioteca `seaborn` para nos ajudar.

Este método permite gerar um gráfico de barras onde podemos informar os valores no eixo x, as categorias no eixo y e os diferentes grupos (disciplinas) pela cor (`hue`).

Para isso, no entanto, precisamos converter nosso dataframe para o formato longo, o que fazemos usando o método `unstack`.


In [0]:
dados_obrigatórias = percentual_obrigatórias.unstack().reset_index(name="percentual")
dados_obrigatórias

In [0]:
dados_obrigatórias.to_csv("/content/drive/My Drive/obrigatorias-imd-2014-2018.csv", index=False)

Para melhorar a legibilidade do gráfico, configuramos também sua proporção e legenda:

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [0]:
sns.catplot(x="percentual", y="descricao", hue="nome", kind="bar", data=dados_obrigatórias, legend=True, legend_out=True, height=20, aspect=0.6)
plt.xlabel("Percentual")
plt.ylabel("Resultado")
plt.title("Resultado dos alunos por disciplina")

Dada a elevada quantidade de disciplinas, se torna difícil enxergar padrões sem o auxílio de algoritmos apropriados para isso.

Por hora, vamos visualizar a quantidade de vezes que cada discente se matriculou em cada disciplina:

In [0]:
tentativas_obrigatórias = matrículas_obrigatórias.groupby(["nome","discente"]).size()
tentativas_obrigatórias

Convertendo a série em dataframe:

In [0]:
dados_tentativas = tentativas_obrigatórias.reset_index(name="tentativas")
dados_tentativas

Para analisarmos as distribuições de tentativas por disciplina, vamos usar boxplots e histogramas.

In [0]:
plt.figure(figsize=(6,8))
sns.boxplot(x="tentativas", y="nome", data=dados_tentativas)
plt.xlabel("Número de tentativas")

Analisando inicialmente os boxplots acima, vemos claramente dois grupos de disciplinas:
1. disciplinas em que apenas outliers precisam tentar mais de uma vez
2. disciplinas em que uma parte significativa dos discentes precisa de mais de uma tentativa

Aqui cabe uma ressalva: as disciplinas Pensamento computacional, Matemática elementar, Análise combinatória e Geometria euclidiana foram criadas em 2018. 

Mesmo com poucos dados, já dá para perceber a mesma diferenciação entre Geometria euclidiana e as demais.

Vamos dar uma olhada nos histogramas para ver em maior detalhe a distribuição de tentativas por disciplina:

In [0]:
dados_tentativas.hist(by="nome", column="tentativas", figsize=(20,12), bins=6, range=(1,6))
plt.xlabel("Número de tentativas")
plt.ylabel("Número de discentes")

Assim como indicado pelos boxplots, podemos ver dois grandes padrões entre as distribuições.

No entanto, ainda que as caudas das distribuições não sejam realçadas pelos boxplots, as disciplinas com múltiplas tentativas apresentam uma grande quantidade de discentes tentando 3 ou mais vezes.

## Analisando cancelamentos na UFRN

Um dado que chama a atenção na análise anterior é a alta quantidade de cancelamentos nas disciplinas obrigatórias do BTI.

No contexto da UFRN, um cancelamento ocorre quando um discente solicita o cancelamento de todo o seu semestre letivo, perdendo todas as disciplinas de uma vez.

Originalmente, esse mecanismo foi pensado para atender pessoas que tivessem problemas de saúde, mudança temporária ou qualquer outro motivo que necessitassem até 2 anos de ausência da universidade.

Na prática, esse mecanismo acaba sendo usado quando discentes estão próximos a ser jubilados por insucessos (um discente pode ter no máximo 3 insucessos em uma disciplina).

Vamos então analisar esta situação em cursos de graduação da UFRN em geral:

In [0]:
turmas_graduação = dados_turmas.query('nivel_ensino == "GRADUAÇÃO"')
turmas_graduação

Identificadas as turmas de graduação, vamos cruzar os dados com os componentes e matrículas:

In [0]:
componentes_turmas = pd.merge(turmas_graduação, componentes, left_on="id_componente_curricular", right_on="id_componente")
componentes_turmas

In [0]:
matrículas_ufrn = pd.merge(dados_matrículas, componentes_turmas, on="id_turma")
matrículas_ufrn

Agora que cruzamos os dados, vamos verificar os resultados possíveis dos discentes:

In [0]:
matrículas_ufrn.descricao.value_counts()

Para podermos comparar com os dados que observamos na análise do BTI, precisamos restringir esses resultados aos casos que usamos naquela análise:

In [0]:
lista_não_cursou_ufrn = [
                        "AGUARDANDO DEFERIMENTO",
                        "EM ESPERA",
                        "INCORPORADO",
                        "MATRICULADO",
                        "REPROVADO POR NOTA",
                        "REPROVADO POR NOTA E FALTA",
                        "REPROVADO POR FALTAS",
                        "TRANSFERIDO"
                        ]
condições_a_remover = lista_não_cursou + lista_não_cursou_ufrn
condição_tentativa_ufrn = f"not descricao in {condições_a_remover}"
matrículas_ufrn = matrículas_ufrn.query(condição_tentativa_ufrn)
matrículas_ufrn.descricao.value_counts()

Agora que filtramos os dados, vamos agregá-los por unidade e descrição:

In [0]:
percentual_ufrn = pd.crosstab(matrículas_ufrn["unidade_responsavel"], matrículas_ufrn["descricao"], normalize="index")
percentual_ufrn

Como nosso interesse é apenas nos casos cancelados, vamos filtrar os dados e ordená-los:

In [0]:
cancelados_ufrn = percentual_ufrn["CANCELADO"].sort_values()
cancelados_ufrn

😱

A proporção entre os dados de mínimo e máximo é de 230 vezes!

Bom, extremos podem ser enganosos, então vamos dar uma olhada na distribuição dos dados:

In [0]:
from scipy.stats import norm

In [0]:
sns.distplot(cancelados_ufrn, fit=norm, bins=20)
plt.xlabel("Percentual")
plt.ylabel("Distribuição")
plt.title("Quantidade de cancelamentos por unidade acadêmica")

A distribuição dos dados se aproxima de uma distribuição normal, mas há indícios de bimodalidade.

Além disso, dá para ver que há dois conjuntos de outliers nas extremidades dos dados:

In [0]:
cancelados_ufrn[cancelados_ufrn <= 0.01]

O conjunto de outliers à esquerda do gráfico é composto predominantemente por unidades relacionadas à medicina.

Por sua vez, os outliers à direita são todos relacionados à ciências exatas:

In [0]:
cancelados_ufrn[cancelados_ufrn >= 0.1]

Para concluir, vamos dar uma olhada nas estatísticas descritivas dessa série:

In [0]:
cancelados_ufrn.describe()

Tanto a média como a mediana estão próximas a 6%.

No entanto, o terceiro quartil é o triplo do primeiro quartil.

Vamos ver os cursos em cada região:

In [0]:
cancelados_ufrn[cancelados_ufrn <= cancelados_ufrn.quantile(0.25)]

Aqui nós já vemos uma mistura maior de cursos, mas a presença de unidades relacionadas à medicina continua elevada.

In [0]:
cancelados_ufrn[cancelados_ufrn >= cancelados_ufrn.quantile(0.75)]

Neste caso, vemos que taxas de cancelamento próximas a 10% são comuns em diferentes áreas da UFRN, envolvendo ciências humanas, biociências e exatas.